<a href="https://colab.research.google.com/github/nsp8/Machine-Learning-Resources/blob/colab-ml-practice/FloodPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e5:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73278%2F8121328%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240601%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240601T000026Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1a828a1ed098bff8700cbc72f59394d8a3488f06daa8097e503ba0342210d6c480ee69ac1b193f3b29a0c279875b9583c1835af70a1e3b3882ef18b8ab6fa70cfb2fa90bec1557afdd4eaf3423773d8269898bccfd22ef58adc19bd7115c97913f98fcc012134894b9d7dad59a876bad8e1cd02145fd42488f38c4b45dc2cf40a67d70035d8b0470f00eb5d1eff447b2e46199840f43b3bdd27950bf4012bb9226d3faa88d6e5da0bc0c96b5d451cdd05940a647ff350dab17b306db42d7ee008622564a2bf197abcf5fb0202c60e8a048a5099345037c8f126b8e6a5d650956bf94648db05a999eaf0ec09510464624caea6071e376e69e26fad0d701b27c9e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Imports**

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor

## **Data loading**

In [ ]:
base_folder_path = "/kaggle/input/playground-series-s4e5"
train_data = pd.read_csv(os.path.join(base_folder_path, "train.csv"))
test_data = pd.read_csv(os.path.join(base_folder_path, "test.csv"))

## **Data Exploration**

In [ ]:
train_data.describe()

In [ ]:
# No null counts?
print(f"No nulls, right? \n> {all(train_data.isnull().sum() == 0)}")

In [ ]:
# Data type information
train_data.info()

In [ ]:
# Layout of data
features_train = train_data.set_index("id")
target_column = "FloodProbability"
label_train = features_train.pop(target_column)
features_train.hist(bins=15, figsize=(20, 45), layout=(4, 5))

> Looks like well distributed data

### Visualizing standardized training data

Standardizing data just for sanity

In [ ]:
def standardize(data):
    standard_scaler = StandardScaler()
    standard_scaler.fit(data)
    return standard_scaler.transform(data)

features_train_standardized = pd.DataFrame(standardize(features_train), columns=features_train.columns)
features_train_standardized.hist(bins=15, figsize=(20, 45), layout=(4, 5))

> The data was already pretty standardized too.

In [ ]:
# Repeating for the target
def standardize_labels(label_data):
    mean, std = label_data.mean(), label_data.std()
    return label_data.apply(lambda v: (v - mean) / std)


label_train_standardized = pd.DataFrame(standardize_labels(label_train), columns=[target_column])
label_train_standardized.hist(bins=15, figsize=(3, 6))

> wait, isn't this probability?

### Scaling the target column to a [0, 1] range to represent probability

In [ ]:
target_scaler = MinMaxScaler()
target_scaler.fit(label_train_standardized)
label_train_transformed = target_scaler.transform(label_train_standardized)
label_train_transformed = pd.DataFrame(label_train_transformed, columns=[target_column])
label_train_transformed.hist(bins=15, figsize=(3, 6))

> this looks much better

### Splitting data into training and testing datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features_train_standardized, label_train_transformed, test_size=0.25
)

In [ ]:
# verifying the columns
X_train.columns

#### **Taking a look at the heatmap of the feature space**

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(X_train.corr(), vmax=1, square=True, annot=True, cmap="plasma")

## **Feature Analyzer**

Analyzing features:

In [ ]:
class FeatureAnalyzer:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.pca = PCA()
        self.pca.fit_transform(self.X)

    @property
    def X_pca(self):
        x_pca = self.pca.fit_transform(self.X)
        return pd.DataFrame(x_pca, columns=[f"PC{i+1}" for i in range(x_pca.shape[1])])

    @property
    def pca_loadings(self):
        return pd.DataFrame(self.pca.components_.T, columns=self.X_pca.columns, index=self.X.columns)

    def plot_variance(self, width=8, dpi=100):
        fig, axes = plt.subplots(1, 2)
        n = self.pca.n_components_
        print(f"\n{'-' * 35}\nPCA Loadings \n{'-' * 35}\n")
        print(self.pca_loadings)
        grid = np.arange(1, n+1)
        explained_variance = self.pca.explained_variance_ratio_
        axes[0].bar(grid, explained_variance)
        axes[0].set(xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0))
        cumulative_variance = np.cumsum(explained_variance)
        axes[1].plot(np.r_[0, grid], np.r_[0, cumulative_variance], marker="h")
        axes[1].set(xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0))
        fig.set(figwidth=width, dpi=dpi)
        return axes

    def make_mi_scores(self, discrete_features: bool = False):
        mi_regression = mutual_info_regression(self.X, self.y, discrete_features=discrete_features)
        return pd.Series(mi_regression, name="Mutual Info Scores", index=self.X.columns).sort_values(ascending=False)

    def check_outliers(self):
        sns.catplot(
            y="value",
            col="variable",
            data=self.X_pca.melt(),
            kind="boxen",
            sharey=False,
            col_wrap=2
        )


def show_feature_analysis_artifacts(X, y, show_mi_scores=True, show_outliers=True):
    feature_analyzer = FeatureAnalyzer(X, y)
    feature_analyzer.plot_variance()
    if show_mi_scores:
        mi_scores = feature_analyzer.make_mi_scores()
        print(f"\n{'-' * 35}\nMutual Information Scores \n{'-' * 35}\n")
        print(mi_scores)
    if show_outliers:
        feature_analyzer.check_outliers()
    return feature_analyzer


feature_analyzer_object = show_feature_analysis_artifacts(
    X_train, y_train, show_mi_scores=False, show_outliers=False
)

## **Visualization for PCA**

In [ ]:
class Charts:
    def __init__(self, data, width=18, height=9, font_family="Sans Serif"):
        self.fig, self.ax = plt.subplots(figsize=(width, height))
        self.data = data
        self.y_pos = [i * 0.9 for i in range(len(data))]
        self.ax.barh(
            self.y_pos,
            self.data.values(),
            height=0.55,
            align="edge",
            color="blue",
        )
        self.font_family = font_family

    @property
    def max_data_value(self):
        return max(self.data.values())

    @property
    def min_data_value(self):
        return min(self.data.values())

    @property
    def axis_period(self):
        return int(self.max_data_value) // 10 or 1

    def set_axes(self):
        from math import ceil
        data_size = len(self.data)
        num_ticks = ceil(self.max_data_value)
        self.ax.xaxis.set_ticks([i * self.axis_period for i in range(num_ticks)])
        self.ax.xaxis.set_ticklabels(
            [i * self.axis_period for i in range(0, num_ticks)],
            size=16,
            fontfamily=self.font_family,
            fontweight=100,
        )
        self.ax.xaxis.set_tick_params(labelbottom=False, labeltop=True, length=0)
        self.ax.set_axisbelow(True)
        self.ax.grid(axis="x", color="#8C92AC", lw=1.2)
        self.ax.spines["left"].set_visible(False)
        self.ax.spines["right"].set_visible(False)
        self.ax.spines["top"].set_visible(False)
        self.ax.spines["bottom"].set_visible(False)
        self.ax.spines["left"].set_lw(1.5)
        self.ax.spines["left"].set_capstyle("butt")
        self.ax.yaxis.set_visible(False)
        return self

    def add_bar_text(self, label_object: dict):
        self.ax.text(
            x=label_object["x_pos"],
            y=label_object["y_pos"],
            s=label_object["text"],
            c=label_object["color"],
            fontfamily=self.font_family,
            fontsize=12,
            va="center",
            path_effects=label_object["path_effects"],
        )

    def add_labels(self, padding: float = 0.3):
        offset = 0.04
        for label, value, y_pos in zip(
            self.data.keys(), self.data.values(), self.y_pos
        ):
            metric_label = dict(
                x_pos=int(self.min_data_value) + (padding * 2.2),
                y_pos=(y_pos + 0.25),
                text=label,
                color="white",
                path_effects=[withStroke(linewidth=6, foreground="green")]
            )
            value_label = dict(
                x_pos=value,
                y_pos=(y_pos + 0.25),
                text=f"{round(value, 3)}0",
                color="green",
                path_effects=[withStroke(linewidth=6, foreground="white")]
            )
            if value < 0:
                metric_label["x_pos"] = self.min_data_value - (padding * 0.7) - offset
                metric_label["path_effects"] = [withStroke(linewidth=6, foreground="red")]
                metric_label["color"] = "white"
                value_label["color"] = "red"
                value_label["x_pos"] -= offset
            self.add_bar_text(metric_label)
            self.add_bar_text(value_label)

        return self

    def add_title(self, title, pos=0.5):
        self.fig.text(
            pos, 0.925, title, fontsize=16, fontweight="bold", fontfamily=self.font_family
        )
        return self

    def plot(self, title=None):
        self.set_axes().add_labels()
        if title:
            self.add_title(title)
        plt.show()


Plotting the first principal component

In [ ]:
def plot_principal_component(feature_analyzer_object, level=1):
    if level < 1:
        raise ValueError("PC level can't be less than 1")
    _loadings = feature_analyzer_object.pca_loadings
    _column = _loadings.columns[level - 1]
    pc_dataframe = _loadings.loc[:, _column].reset_index().to_dict(orient="records")
    feature_to_pc1 = {_entry["index"]: float(_entry[_column]) for _entry in pc_dataframe}
    chart = Charts(data=feature_to_pc1, font_family="monospace")
    chart.plot(title=_column)

plot_principal_component(feature_analyzer_object, level=1)

#### **Enhancing the features based on the PCA analysis**

In [ ]:
X_train_pca = feature_analyzer_object.pca.fit_transform(X_train)
X_test_pca = feature_analyzer_object.pca.transform(X_test)

## **Modeling**

In [ ]:
from collections import defaultdict

models = defaultdict(list)

### **Trying the RandomForestRegressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor


class RandomForestRegression:
    def __init__(self, train_data, test_data, feature_list, target, num_trees=10, criterion="absolute_error"):
        self.regressor = RandomForestRegressor(
            n_estimators=num_trees,
            criterion=criterion,
            n_jobs=-1
        )
        self.train_data = train_data
        self.test_data = test_data
        self.feature_list = feature_list
        self.target = target

    def fit(self):
        train_features, train_labels = self.train_data
        self.regressor.fit(train_features, train_labels)
        return self

    def predict(self):
        test_features, test_labels = self.test_data
        assert isinstance(test_features, np.ndarray)
        assert isinstance(test_labels, pd.DataFrame)
        dataframe = pd.DataFrame(test_features, columns=self.feature_list)
        dataframe[self.target] = test_labels
        dataframe["predictions"] = self.regressor.predict(test_features)
        self.prediction_data = dataframe
        return self

    def set_metrics(self):
        y_pred, y_true = self.prediction_data["predictions"], self.prediction_data[self.target]
        self.r2_score = r2_score(y_true, y_pred)

In [ ]:
rf_regressor_model = RandomForestRegression(
    train_data=(X_train_pca, y_train),
    test_data=(X_test_pca, y_test),
    feature_list=X_train.columns,
    target=target_column,
    num_trees=5
)
# rf_regressor_model.fit().train().set_metrics()

> this took forever to converge to a result; * *sigh* * moving on...

### **XGBoost Regressor**

#### Training the XGBRegressor model:

In [ ]:
xgbregressor_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    early_stopping_rounds=500,
    n_jobs=5,
    reg_lambda=1e-4,
    tree_method="hist",
    eval_metric=r2_score
)
xgbregressor_model.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)], verbose=False)

#### Calculating the predictions:

In [ ]:
predictions_valid = xgbregressor_model.predict(X_test_pca)

#### Checking the error metric

In [ ]:
from sklearn.metrics import mean_squared_error

print(f"Validation set prediction error: {mean_squared_error(y_test, predictions_valid):.4f}")

#### Making final predictions

In [ ]:
final_predictions = xgbregressor_model.predict(features_train_standardized)
print(f"Test set prediction error: {mean_squared_error(label_train_transformed, final_predictions):.4f}")

### **Conveying the $R^2$ score**

In [ ]:
r2_score_max = max(xgbregressor_model.evals_result_["validation_0"]["r2_score"])
print(f"R2 Score: {r2_score_max}")

## **Saving the output**

Storing the result in the submission file

In [ ]:
output_df = pd.DataFrame(final_predictions, columns=[target_column], index=label_train_transformed.index).reset_index().rename(columns={"index": "id"})
output_df.to_csv("submission.csv", index=False)